## ETL (Extract, Transform and Load)

Importamos las librerias necesarias.

In [1]:
import ast
import pandas as pd
import numpy as np

Leemos el archivo de juegos de steam y para analizar la estructura, solo leeremos las primeras filas

In [2]:
rows = []
with open ('_src/Data/steam_games.json') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
df=pd.DataFrame(rows)
df.head(2)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN


Comenzaremos con la primera columna que es: publisher

In [3]:
#Contamos la cantidad de valores NaN en la columna "publisher"
num_nan_publisher = df["publisher"].isna().sum()
print(num_nan_publisher)
# Cambiamos los valores NaN por "Desconocido" en la columna "publisher"
df["publisher"].fillna("Desconocido", inplace=True)
num_nan_editora = df["publisher"].isna().sum()
print(num_nan_editora)

8052
0


In [4]:
print(all(isinstance(x, str) for x in df['publisher']))
print(df['publisher'].apply(type).value_counts())  #Nos aseguramos de que los valores de la columan sean 'str'

True
publisher
<class 'str'>    32135
Name: count, dtype: int64


Después de realizar un análisis, las columnas _`url`_ y _`reviews_url`_ no serán relevantes para nuestro desarrollo, 
así que, la eliminaremos del df

In [5]:
df.drop(columns=['url', 'reviews_url'], inplace=True)

Ahora revisaremos la columna genres

In [6]:
# Contar la cantidad de valores NaN en la columna "genres"
num_nan_genres = df["genres"].isna().sum()
print(num_nan_genres)

3283


In [7]:
print(all(isinstance(x, str) for x in df['genres']))
print(df['genres'].apply(type).value_counts()) #dentro de la columna "genres", tenemos listas
#por ahora no lo voy a desanidar, porque para las primeras 6 funciones que me piden, no es necesario desanidarlos

False
genres
<class 'list'>     28852
<class 'float'>     3283
Name: count, dtype: int64


Si revisamos _`tags`_, siempre se repiten los géneros y para simplificar el modelo, será mejor que lo eliminemos 

In [8]:
df.drop(columns=['tags'], inplace=True)

Ahora trabajaremos con las columnas 'app_name' y 'title'

In [9]:
# Contar la cantidad de valores NaN en la columna "app_name"
num_nan_nombre_app = df["app_name"].isna().sum()
num_nan_nombre_app

2

In [10]:
# Contar la cantidad de valores NaN en la columna "title"
num_nan_titulo = df["title"].isna().sum()
num_nan_titulo

2050

In [11]:
# Ya que los valores de title tiene muchos NaNs, usaremos app_name y eliminaremos titulo
df.drop(columns=['title'], inplace=True)

In [12]:
#cambiar los valores faltantes de app_name con Desconocido
df["app_name"].fillna("Desconocido", inplace=True)

In [13]:
print(all(isinstance(x, str) for x in df['app_name']))
print(df['app_name'].apply(type).value_counts())

True
app_name
<class 'str'>    32135
Name: count, dtype: int64


Eliminaremos la columna id ya que será suficiente que trabajemos con la columna app_name

In [14]:
df.drop(columns=['id'], inplace=True)

Cambiar los valores NaN por "Desconocido" en la columna _`developer`_

In [15]:
df["developer"].fillna("Desconocido", inplace=True)

In [16]:
# Contar la cantidad de valores NaN en la columna "developer"
num_nan_developer = df["developer"].isna().sum()
print(num_nan_developer) #Ahora ya no tengo NaN en la columna desarrollador

0


In [17]:
print(all(isinstance(x, str) for x in df['developer']))
print(df['developer'].apply(type).value_counts())

True
developer
<class 'str'>    32135
Name: count, dtype: int64


Ahora analizaré la columna price

In [18]:
num_nan_precio = df["price"].isna().sum()
num_nan_precio

1377

In [19]:
unique_precios = df['price'].unique()
print(unique_precios)

[4.99 'Free To Play' 'Free to Play' 0.99 2.99 3.99 9.99 18.99 29.99 nan
 'Free' 10.99 1.59 14.99 1.99 59.99 8.99 6.99 7.99 39.99 19.99 7.49 12.99
 5.99 2.49 15.99 1.25 24.99 17.99 61.99 3.49 11.99 13.99 'Free Demo'
 'Play for Free!' 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99 79.99
 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71 'Install Now'
 21.99 89.99 'Play WARMACHINE: Tactics Demo' 0.98 139.92 4.29 64.99
 'Free Mod' 54.99 74.99 'Install Theme' 0.89 'Third-party' 0.5 'Play Now'
 299.99 1.29 3.0 15.0 5.49 23.99 49.0 20.99 10.93 1.39
 'Free HITMAN™ Holiday Pack' 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5 199.0
 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0 40.0
 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2.89 'Play the Demo' 99.0
 87.94 599.0 8.98 9.69 0.49 9.98 9.95 7.0 'Starting at $499.00'
 'Starting at $449.00' 12.8

In [20]:
#reemplazar algunos valores que no están en formato float
df['price'] = df['price'].replace('Starting at $499.00', 499.0)
df['price'] = df['price'].replace('Starting at $449.00', 449.0)

In [21]:
#reemplazar en price todo lo que diga Free con 0.0
df['price'] = df['price'].replace('Free to Play', 0.0)
df['price'] = df['price'].replace('Free To Play', 0.0)
df['price'] = df['price'].replace('Free', 0.0)
df['price'] = df['price'].replace('Free Demo', 0.0)
df['price'] = df['price'].replace('Play for Free!', 0.0)
df['price'] = df['price'].replace('Play WARMACHINE: Tactics Demo', 0.0)
df['price'] = df['price'].replace('Free Mod', 0.0)
df['price'] = df['price'].replace('Free HITMAN™ Holiday Pack', 0.0)
df['price'] = df['price'].replace('Play the Demo', 0.0)
df['price'] = df['price'].replace('Free to Try', 0.0)
df['price'] = df['price'].replace('Free Movie', 0.0)
df['price'] = df['price'].replace('Free to Use', 0.0)

In [22]:
print(df['price'].apply(type).value_counts())

price
<class 'float'>    32129
<class 'str'>          6
Name: count, dtype: int64


In [23]:
unique_precios = df['price'].unique()
print(unique_precios)

[4.99 0.0 0.99 2.99 3.99 9.99 18.99 29.99 nan 10.99 1.59 14.99 1.99 59.99
 8.99 6.99 7.99 39.99 19.99 7.49 12.99 5.99 2.49 15.99 1.25 24.99 17.99
 61.99 3.49 11.99 13.99 34.99 74.76 1.49 32.99 99.99 14.95 69.99 16.99
 79.99 49.99 5.0 44.99 13.98 29.96 119.99 109.99 149.99 771.71
 'Install Now' 21.99 89.99 0.98 139.92 4.29 64.99 54.99 74.99
 'Install Theme' 0.89 'Third-party' 0.5 'Play Now' 299.99 1.29 3.0 15.0
 5.49 23.99 49.0 20.99 10.93 1.39 36.99 4.49 2.0 4.0 9.0 234.99 1.95 1.5
 199.0 189.0 6.66 27.99 10.49 129.99 179.0 26.99 399.99 31.99 399.0 20.0
 40.0 3.33 199.99 22.99 320.0 38.85 71.7 59.95 995.0 27.49 3.39 6.0 19.95
 499.99 16.06 4.68 131.4 44.98 202.76 1.0 2.3 0.95 172.24 249.99 2.97
 10.96 10.0 30.0 2.66 6.48 19.29 11.15 18.9 2.89 99.0 87.94 599.0 8.98
 9.69 0.49 9.98 9.95 7.0 499.0 449.0 12.89 6.49 1.87 42.99 41.99 289.99
 23.96 5.65 12.0 13.37 189.96 124.99 19.98 160.91]


In [24]:
#como no sabemos los precios, eliminaremos estas filas para que no altere nuestro análisis. A demás, son poquísimas
print(df['price'].value_counts().get('Install Now', 0))
print(df['price'].value_counts().get('Install Theme', 0))
print(df['price'].value_counts().get('Third-party', 0))
print(df['price'].value_counts().get('Play Now', 0)) 

1
1
2
2


In [25]:
# Eliminar filas con los valores 'Install Now', 'Install Theme','Third-party' y 'Play Now' en la columna 'price'
df = df.drop(df[df['price'].isin(['Install Now', 'Install Theme','Third-party', 'Play Now'])].index)

In [26]:
print(all(isinstance(x, float) for x in df['price']))
print(df['price'].apply(type).value_counts())

True
price
<class 'float'>    32129
Name: count, dtype: int64


Revisar discount_price

In [27]:
unique_precios = df['discount_price'].unique()
print(unique_precios)

[  4.49    nan   0.83   1.79   8.79   1.59   1.97   0.59   0.89   0.84
   7.19   6.79   9.89   5.99  14.99   0.49   0.74  22.66  49.96   6.29
   3.14  59.11   7.49  13.39  10.49  35.97   0.99   7.99   4.99  11.39
   6.99  24.9    3.59   2.69   3.49   4.68   4.19  17.08   8.99   0.62
   4.89   2.99  19.78   2.05   2.39  19.99  79.99  49.99 139.99   2.44
   0.69   3.74   0.79   1.49   3.99   2.09  10.04  24.82  29.75   6.74
   4.79   2.19   3.34   0.5    5.24   2.51   1.19   1.99   0.66   3.24
  22.46   9.99  44.1   17.49   1.39  31.49   1.69   5.39   4.24  25.49
   3.19  11.69  11.99]


In [28]:
print(df['discount_price'].isna().sum())

31904


In [29]:
print(all(isinstance(x, float) for x in df['discount_price']))
print(df['discount_price'].apply(type).value_counts())

True
discount_price
<class 'float'>    32129
Name: count, dtype: int64


En la columna _`sentiment`_ realizaremos algunos cambios porque los datos no están muy bien categorizados

In [30]:
df['sentiment'].unique()

array([nan, 'Mostly Positive', 'Mixed', '1 user reviews',
       '3 user reviews', '8 user reviews', 'Very Positive',
       'Overwhelmingly Positive', '6 user reviews', '5 user reviews',
       '2 user reviews', 'Very Negative', 'Positive', 'Mostly Negative',
       '9 user reviews', 'Negative', '4 user reviews', '7 user reviews',
       'Overwhelmingly Negative'], dtype=object)

In [31]:
#Cambiar los que no tienen una clasificación a "sin clasificar"
df['sentiment'] = df['sentiment'].str.replace(r'\d+ user reviews', 'sin clasificar', regex=True)
df['sentiment'] = df['sentiment'].fillna('sin clasificar')

In [32]:
df['sentiment'].unique()

array(['sin clasificar', 'Mostly Positive', 'Mixed', 'Very Positive',
       'Overwhelmingly Positive', 'Very Negative', 'Positive',
       'Mostly Negative', 'Negative', 'Overwhelmingly Negative'],
      dtype=object)

In [33]:
print(all(isinstance(x, str) for x in df['sentiment']))
print(df['sentiment'].apply(type).value_counts())

True
sentiment
<class 'str'>    32129
Name: count, dtype: int64


Continuamos con la columna metascore

In [34]:
df['metascore'].unique()

array([nan, 96, 84, 80, 76, 70, 'NA', 69, 81, 75, 72, 66, 67, 77, 91, 89,
       83, 61, 88, 65, 94, 57, 86, 87, 92, 79, 82, 58, 74, 85, 90, 68, 71,
       60, 73, 59, 64, 54, 53, 78, 51, 44, 63, 38, 56, 49, 52, 62, 93, 48,
       34, 95, 43, 55, 24, 46, 41, 20, 39, 45, 35, 47, 40, 36, 50, 32, 37,
       33, 42, 27, 29, 30], dtype=object)

In [35]:
import numpy as np

# Reemplazar 'NA' con NaN
df['metascore'].replace('NA', np.nan, inplace=True)

In [36]:
df['metascore'].unique()

array([nan, 96., 84., 80., 76., 70., 69., 81., 75., 72., 66., 67., 77.,
       91., 89., 83., 61., 88., 65., 94., 57., 86., 87., 92., 79., 82.,
       58., 74., 85., 90., 68., 71., 60., 73., 59., 64., 54., 53., 78.,
       51., 44., 63., 38., 56., 49., 52., 62., 93., 48., 34., 95., 43.,
       55., 24., 46., 41., 20., 39., 45., 35., 47., 40., 36., 50., 32.,
       37., 33., 42., 27., 29., 30.])

In [37]:
print(all(isinstance(x, float) for x in df['metascore']))
print(df['metascore'].apply(type).value_counts())

True
metascore
<class 'float'>    32129
Name: count, dtype: int64


Realizamos modificaciones en la columna _`release_date`_

In [38]:
# Convertimos la columna 'release_date' al formato de fecha
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

In [39]:
# Crear una nueva columna 'year' que contiene solo el año de la fecha
df['year'] = df['release_date'].dt.strftime('%Y')

In [40]:
#Eliminar la columna "release_date"
df.drop(columns=['release_date'], inplace=True)

In [41]:
df['year'].apply(type).value_counts()

year
<class 'str'>      29777
<class 'float'>     2352
Name: count, dtype: int64

In [42]:
df['year'].isna().sum() #hay 2352 registros nulos en la fila year

2352

Como todas las funciones que piden hacer está en base al **año**, es mejor eliminar los registros NaN, porque no van a alterar el resultado

In [43]:
df.dropna(subset=['year'], inplace=True)

Convertimos las columnas _`genres`_ y _`specs`_ a cadena

In [44]:
# Reemplaza valores nulos por cadena vacia
df['genres'].fillna('', inplace=True)

# Convierte los valores en la columna 'genres' a cadena
df['genres'] = df['genres'].apply(', '.join)

In [45]:
# Reemplaza valores nulos por cadena vacia
df['specs'].fillna('', inplace=True)

# Convierte los valores en la columna 'specs' a cadena
df['specs'] = df['specs'].apply(', '.join)

Volvemos a colocar los valores nulos a _`specs`_ y _`genres`_

In [46]:

df['specs'] = df['specs'].apply(lambda x: None if x == '' else x)
df['genres'] = df['genres'].apply(lambda x: None if x == '' else x)

In [47]:
df.shape #Al final solo nos quedamos con 11 columanas y 29777 registros

(29777, 11)

Importaremos el archivo en formato **parquet** porque pesa menos

In [48]:
df.to_parquet('_src/Data/steam_games_transform.parquet')